##### ✅ `آخرین بروزرسانی در : 20-10-2025`

#  `مجوزهای مورد نیاز `

## 🔑 ابتدا این دو مجوز را **یکبار** تأیید کنید  
> ⚠️ توجه: فقط بار اول نیاز به تأیید دارند، دفعات بعدی دیگر نیازی نیست.  

---

### 📝 مجوز اول  
👉 [pyannote/speaker-diarization](https://huggingface.co/pyannote/speaker-diarization)  

### 📝 مجوز دوم  
👉 [pyannote/segmentation](https://huggingface.co/pyannote/segmentation)  

---

✅ بعد از تأیید این دو مورد، می‌توانید اجرای پروژه را شروع کنید.  


In [1]:
# @title Install requirements for SoniTranslate
#@markdown ###`نصب کتابخانه ها`

from IPython.display import display, HTML

message = """
<div style="text-align: center;">
    <p>⚠️ در صورتی که با تایید مجوزها آشنایی ندارید این ویدیو را تا دقیقه 4:44 تماشا کنید.</p>
    <div style="transform: scale(0.5); transform-origin: top left;">
        <iframe width="560" height="315" src="https://www.youtube.com/embed/UgF-ejzCEN4" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
    </div>
</div>
"""
display(HTML(message))

%cd /content

# Clean up previous installations
!rm -rf SoniTranslate
!rm -rf .venv

!git clone https://github.com/r3gm/SoniTranslate.git
%cd SoniTranslate

# Uninstall conflicting packages
!pip uninstall chex pandas-stubs ibis-framework albumentations albucore jax -y -q 2>/dev/null || true

# Install uv and setup environment
!pip install uv==0.8.13 -q
!uv venv --python 3.10 --clear -q

# Setup pip
!curl -sS https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!uv run python get-pip.py pip==23.1.2 -q
!uv run python -m pip install -q pip==23.1.2 setuptools==69.5.1 wheel -U

# Install git-lfs
!apt install git-lfs -qq
!git lfs install

# Install PyTorch first
print("Installing PyTorch...")
!uv run python -m pip install -q torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

# Fix whisperX dependency
!sed -i 's|git+https://github.com/R3gm/whisperX.git@cuda_11_8|git+https://github.com/R3gm/whisperX.git@cuda_12_x|' requirements_base.txt

# Install build dependencies first
print("Installing build dependencies...")
!uv run python -m pip install -q cython numpy==1.26.4

# Install base requirements with error handling
print("Installing base requirements...")
!uv run python -m pip install -q -r requirements_base.txt --no-build-isolation 2>&1 | grep -v "Running setup.py install" || true

# Install extra requirements
print("Installing extra requirements...")
!uv run python -m pip install -q -r requirements_extra.txt 2>&1 | grep -v "Running setup.py install" || true

# Install onnxruntime
print("Installing onnxruntime...")
!uv run python -m pip install -q onnxruntime-gpu==1.22.0

Install_PIPER_TTS = True # @param {type:"boolean"}

if Install_PIPER_TTS:
    print("Installing Piper TTS...")
    !uv run python -m pip install -q piper-tts==1.2.0 2>&1 | grep -v "Running setup.py install" || true

Install_Coqui_XTTS = True # @param {type:"boolean"}

if Install_Coqui_XTTS:
    print("Installing Coqui XTTS...")
    !uv run python -m pip install -q -r requirements_xtts.txt 2>&1 | grep -v "Running setup.py install" || true
    !uv run python -m pip install -q TTS==0.21.1 --no-deps

# Install libcudnn8
print("Installing libcudnn8...")
!sudo apt install -y libcudnn8 -qq 2>&1 | tail -n 5

print("\n=== Installation completed successfully! ===")

/content
Cloning into 'SoniTranslate'...
remote: Enumerating objects: 1215, done.
remote: Counting objects: 100% (676/676), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 1215 (delta 608), reused 527 (delta 527), pack-reused 539 (from 1)
Receiving objects: 100% (1215/1215), 19.87 MiB | 20.70 MiB/s, done.
Resolving deltas: 100% (842/842), done.
/content/SoniTranslate
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 8.7 MB/s eta 0:00:00
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Updated git hooks.
Git LFS initialized.
Installing PyTorch...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title `Whisper دانلود`
#@markdown حتما این سلول اجرا شود

#@markdown This cell must be run.
import os
import logging
from huggingface_hub import snapshot_download

print("--- شروع پاک‌سازی کش ---")
os.system("rm -rf /root/.cache/huggingface/hub")
print("--- کش پاک‌سازی شد ---")

print("--- غیرفعال کردن دانلودرهای XET و HF_TRANSFER ---")
os.environ["HF_HUB_DISABLE_XET_DOWNLOAD"] = "1"
os.environ["HF_HUB_DISABLE_HF_TRANSFER"] = "1"
print("--- دانلودرها غیرفعال شدند ---")

print("--- شروع دانلود دستی مدل‌ها ---")

logging.basicConfig()
logging.getLogger("huggingface_hub").setLevel(logging.INFO)

try:
    TOKENIZER_REPO = "openai/whisper-large-v3"
    print(f"در حال دانلود فایل‌های توکنایزر از {TOKENIZER_REPO}...")
    snapshot_download(
        repo_id=TOKENIZER_REPO,
        allow_patterns=["*.json", "vocabulary.*"],
        local_dir_use_symlinks=False,
        cache_dir="/root/.cache/huggingface/hub"
    )
    print(f"--- دانلود {TOKENIZER_REPO} کامل شد ---")

    MODEL_REPO = "Systran/faster-whisper-large-v3"
    print(f"در حال دانلود فایل model.bin از {MODEL_REPO}...")
    snapshot_download(
        repo_id=MODEL_REPO,
        allow_patterns=["model.bin"],
        local_dir_use_symlinks=False,
        cache_dir="/root/.cache/huggingface/hub"
    )
    print(f"--- دانلود {MODEL_REPO} کامل شد ---")

    print("\n✅ همه مدل‌ها با موفقیت دانلود شدند!")
    print("اکنون می‌توانید سلول 'RUN THE WEB APP' را اجرا کنید.")

except Exception as e:
    print("\n❌ شکست در دانلود دستی")
    print(f"خطا: {e}")
    print("لطفاً محیط Colab را ریستارت کنید و دوباره تلاش کنید.")

In [ ]:
#@title ### RUN THE WEB APP
#@markdown ##### `توکن هاگین فیس را وارد کنید و اجرا کنید`
YOUR_HF_TOKEN = "" #@param {type:'string'}
from google.colab import userdata

%env YOUR_HF_TOKEN=userdata.get('HF_TOKEN')
theme = "Taithrah/Minimal" # @param ["Taithrah/Minimal", "aliabidlabs/pakistan", "gstaff/xkcd", "ParityError/LimeFace", "abidlabs/pakistan", "rottenlittlecreature/Moon_Goblin", "ysharma/llamas", "gradio/dracula_revamped"]
interface_language = "persian" # @param ['arabic', 'azerbaijani', 'chinese_zh_cn', 'english', 'french', 'german', 'hindi', 'indonesian', 'italian', 'japanese', 'korean', 'marathi', 'persian', 'portuguese', 'russian', 'spanish', 'swedish', 'turkish', 'ukrainian', 'vietnamese']
verbosity_level = "info" # @param ["debug", "info", "warning", "error", "critical"]


%cd /content/SoniTranslate
!uv run python app_rvc.py --theme {theme} --verbosity_level {verbosity_level} --language {interface_language} --public_url

`بعد از اجرای سلول بالا ☝ ، لینک رابط گریدیو نمایش داده میشود که باید آن را باز کنید`

Open the `public URL` when it appears


<font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font><font color="#00FF00">─</font><font color="#0066cc">─</font><font color="#FF0000">─</font>

In [ ]:
#@title #####📥 `دانلود اضطراری ویدیوی دوبله شده و زیرنویس ها یکجا بصورت زیپ`
#@markdown #####  در صورت مشکل در دانلود در محیط گریدیو، سلول قبل را متوقف کنید و این سلول  را اجرا کنید

folder_path = '/content/SoniTranslate/outputs'
zip_filename = '/content/outputs.zip'

import shutil
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', folder_path)

from google.colab import files
files.download(zip_filename)
